## **Performance Evaluation of Terapixel Rendering in Cloud (Super)computing**

In [1]:
# Installing the necessary dependencies
!pip install pandasql

In [2]:
# Importing the libraries
import pandas as pd 
import pandasql as ps 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import  coo_matrix

In [3]:
# Reading the datasets
df  = pd.read_csv('gpu.csv')
df1 = pd.read_csv('task-x-y.csv')
df2 = pd.read_csv('application-checkpoints.csv')